# saturation FBA (satFBA) analysis of *i*CH360
The saturation FBA (satFBA) framework [1] is a variant of enzyme-constrained FBA, wherein a fixed enzyme cost per flux is assumed for all metabolic reactions in a model, except for the substrate transporter, for which a complete kinetic law is used (Methods). Since the external substrate concentration is a simple parameter, screening this concentration is equivalent to screening the values of the transporter efficiency.

1. Müller, S., Regensburger, G. & Steuer, R. Resource allocation in metabolic networks: kinetic optimization and approximations by FBA. Biochemical Society Transactions 43, 1195–1200 (2015).

In [1]:
import cobra
import sys
sys.path.append('../../utils')
import importlib
import EC_utils

## Load model

In [2]:
ec_model=cobra.io.read_sbml_model('../../Model/EC_iCH360/EC_iCH360_fitted_kapps.xml')
ec_model.reactions.EX_glc__D_e_bw.upper_bound=1000

#Make a copy of the model where the ATP maintenance constraint has been lifted
ec_model_no_ATPM=ec_model.copy()
ec_model_no_ATPM.reactions.ATPM_fw.lower_bound=0


'' is not a valid SBML 'SId'.


Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-12
Read LP format model from file C:\Users\marco\AppData\Local\Temp\tmpq2el29xa.lp
Reading time = 0.00 seconds
: 305 rows, 1012 columns, 4822 nonzeros


## Run satFBA (the relevant code can be found in the `utils/EC_utils.py`)

In [3]:
importlib.reload(EC_utils)
sat_fba_results_with_ATPM=EC_utils.run_satFBA(ec_model,saturable_reaction_id='GLCptspp_fw',log_spaced=True,sigma=0.446,n=50,saturation_max=0.95)
sat_fba_results_no_ATPM=EC_utils.run_satFBA(ec_model_no_ATPM,saturable_reaction_id='GLCptspp_fw',log_spaced=True,sigma=0.446,n=50,saturation_max=0.95)

100%|██████████| 50/50 [00:15<00:00,  3.31it/s]


Our implementation solves the satFBA problem for a range of uptake transporter saturation values. In order to convert these saturation values into substrate concentration values, a kinetic law for the substrate uptake reaction must be assumed. Here we assume a simple irreversible Michaelis Mentens kinetic law, so that the uptake saturation, $\sigma_{up}$, takes the form:

$$
\sigma_{\rm up}=\frac{\rm [Glc]}{K_{\rm m}+{\rm [Glc]}}
$$

Inverting this equation yields:

$$
{\rm [Glc]}=\frac{K_{\rm m}~\sigma_{\rm up}}{1-\sigma_{\rm up}}
$$

In [5]:
km_up=0.116
saturato2conc=lambda s:km_up*s/(1-s)
sat_fba_results_no_ATPM['substrate_concentration_mM']=saturato2conc(sat_fba_results_no_ATPM['saturation'].values)
sat_fba_results_with_ATPM['substrate_concentration_mM']=saturato2conc(sat_fba_results_with_ATPM['saturation'].values)

# Save data for plotting in the relevant notebook

In [9]:
sat_fba_results_no_ATPM.reset_index().to_csv("out/satFBA_results_no_ATPM.csv",index=False)
sat_fba_results_with_ATPM.reset_index().to_csv("out/satFBA_results_with_ATPM.csv",index=False)